In [1]:
include("modle.jl")
using .Model
using StaticArrays
# using Plots
using LinearAlgebra
# using Makie
using GLMakie 
using LsqFit

In [2]:
# 定义铜的晶格常数（单位：Å）
lattice_constant = 3.61

# 定义铜的FCC晶胞的基矢量
lattice_vectors = (Matrix([
    lattice_constant 0.0 0.0; #a1
    0.0 lattice_constant 0.0; #a2
    0.0 0.0 lattice_constant] #a3
))'

# 定义铜的FCC晶胞中的原子位置（单位：Å）
atom_positions = [
    Vector([0.0, 0.0, 0.0]),
    Vector([0.0, 0.5, 0.5]),
    Vector([0.5, 0.0, 0.5]),
    Vector([0.5, 0.5, 0.0]),
    Vector([1.0, 0.0, 0.0]),
    Vector([0.0, 1.0, 0.0]),
    Vector([0.0, 0.0, 1.0]),
    Vector([0.5, 1.0, 0.5]),
    Vector([1.0, 0.5, 0.5]),
    Vector([0.5, 0.5, 1.0]),
    Vector([1.0, 0.0, 1.0]),
    Vector([1.0, 1.0, 0.0]),
    Vector([0.0, 1.0, 1.0]),
    Vector([1.0, 1.0, 1.0])
] 

# 创建铜的原子列表
atoms = [Atom(pos) for pos in atom_positions]

cell=UnitCell(lattice_vectors,atoms)
cpcell=copycell(cell,3,3,3)
fig=visualize_unitcell_atoms(cpcell)

In [3]:
#lj势能
function lj(r::Float64)
    return 4*(1/r^12-1/r^6)
end
function Flj(r::Vector{Float64})
    rn=norm(r)
    return 24*(2/rn^14-1/rn^8)*r
end

ct=5.0
interaction = Interaction(lj, Flj, ct, 0.1)

x=1:0.001:ct
y=interaction.cutenergy.(x)
lines(x,y)


In [4]:
println(cell_energy(cpcell,interaction))
println(cell_energy(cpcell,interaction))

-32.02658138108412
-32.02658138108412


In [5]:
el=Vector{Float64}([])
lattice_constant = 3.61
cl=1.3:0.01:2.5
en=0
for lattice_constant in cl
    # 定义铜的FCC晶胞的基矢量
    lattice_vectors = (Matrix([
        lattice_constant 0.0 0.0; #a1
        0.0 lattice_constant 0.0; #a2
        0.0 0.0 lattice_constant] #a3
    ))
    # 创建铜的原子列表
    atoms = [Atom(pos) for pos in atom_positions]

    cell=UnitCell(lattice_vectors,atoms)
    cpcell=copycell(cell,5,5,5)
    en=cell_energy0(cpcell,interaction,ifnormalize=true)
    push!(el,en)
end


In [10]:
# 创建一个新的 Figure
fig = Figure()

# 创建一个 Axis，并设置坐标范围
ax = Axis(fig[1, 1], title = "Line Plot", xlabel = "X-Axis", ylabel = "Y-Axis",
          )

lines!(ax,cl,el)
# ylims!(ax, -0.001, 0.001)

fig

In [14]:
vl=cl.^3*(cpcell.copy[1]*cpcell.copy[2]*cpcell.copy[3]);

In [16]:
lines(vl,el)

In [29]:
# 定义 Birch-Murnaghan 方程的函数
function birch_murnaghan(V, p)
    V0, B0, B0_prime, E0 = p
    eta = (V0 ./ V).^(2/3)  # 使用广播运算符 ./
    E = E0 .+ (9 * V0 * B0 / 16) .* ((eta .- 1).^3 .* B0_prime .+ (eta .- 1).^2 .* (6 .- 4 .* eta))
    return E
end

function BMfit(cl,el,p0::Vector{Float64}=[1200.0, 200.0, 4.0, 1000.0])
    fit = curve_fit(birch_murnaghan, vl, el, p0)

    # 拟合结果
    fitted_params = fit.param
    min_lattice_constant = fitted_params[1].^(1/3)/cpcell.copy[1]
    println("Fitted parameters:")
    println("V0 = ", fitted_params[1])
    println("B0 = ", fitted_params[2])
    println("B0' = ", fitted_params[3])
    println("E0 = ", fitted_params[4])
    println("latticeconstant = ",min_lattice_constant )
    fig = Figure()
    fitfunc(x)=birch_murnaghan(x, fitted_params)
    # 创建一个 Axis，并设置坐标范围
    ax = Axis(fig[1, 1],xlabel="Volume", ylabel="Energy", title="Birch-Murnaghan EOS Fit"
              )
    scatter!(ax,vl,el,color=:red,label="Data")
    lines!(ax,vl,fitfunc.(vl),color=:blue,label="Fit")
    axislegend(ax, position=:rt)
    fig
    return min_lattice_constant
end

BMfit (generic function with 2 methods)

In [30]:
min_BMfunction(el,vl)

Fitted parameters:
V0 = 482.29812570328926
B0 = 144.9893298745432
B0' = 6.4884045087519775
E0 = -8872.44989358147
latticeconstant = 1.568442205842431
Fitted energies: [1.9171861820679873e8, 1.5659637789418423e8, 1.3008722919836451e8, 1.096106748473637e8, 9.348155768588084e7]


MakieCore.InvalidAttributeError: Invalid attribute c for plot type Scatter{Tuple{Vector{Point{2, Float64}}}}.

The available plot attributes for Scatter{Tuple{Vector{Point{2, Float64}}}} are:

alpha        depthsorting     inspector_hover  nan_color         transformation 
clip_planes  distancefield    inspector_label  overdraw          transparency   
color        fxaa             lowclip          rotation          uv_offset_width
colormap     glowcolor        marker           space             visible        
colorrange   glowwidth        marker_offset    ssao                             
colorscale   highclip         markersize       strokecolor                      
cycle        inspectable      markerspace      strokewidth                      
depth_shift  inspector_clear  model            transform_marker                 


Generic attributes are:

clip_planes  dim_conversions  model      transformation  yautolimits        
cycle        label            rasterize  xautolimits   